In [ ]:
# default_exp projects.robustness_benchmark

# Robustness Benchmark
> Benchmark utility code

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export
import pandas as pd
import numpy as np

from enum import Enum 
from functools import partial
from scp.analysis.binary import performance
from scp.utils.dict import apply_to_vals

## Constants

In [ ]:
# export
Corruption = Enum("Corruption", 
                  "gaussian_noise, shot_noise, impulse_noise,\
                   defocus_blur, motion_blur, zoom_blur,\
                   black_corner, char,\
                   brightness_up, brightness_down, contrast, elastic_transform, pixelate, jpeg_compression,\
                   speckle_noise, gaussian_blur, bubble, saturate,\
                   sanity_check")

Perturbation = Enum("Perturbation",
                    "gaussian_noise, shot_noise,\
                     motion_blur, zoom_blur,\
                     char\
                     brightness, translate, rotate, tilt, scale,\
                     speckle_noise, gaussian_blur, bubble, shear,\
                     sanity_check")

max_severity = 6 # severity ranges from 1 to 5 inclusive
num_frames = 31 # 
classes = ["nevus", "melanoma"]



## Analysis

In [ ]:
# export
# get balanced error rate
def bal_error_rate(v):
    perf = performance(v[1], v[0][:,1], bal_err_rate=True)
    return perf["bal_err_rate"]

# get normal error rate
def error_rate(v):
    perf = performance(v[1], v[0][:,1], err_rate=True)
    return perf["err_rate"]

# get auroc
def auroc(v):
    perf = performance(v[1], v[0][:,1], auroc=True)
    return perf["auroc"]

# get acc
def acc(v):
    perf = performance(v[1], v[0][:,1])
    return perf["acc"]

# get spec
def spec(v):
    perf = performance(v[1], v[0][:,1])
    return perf["spec"]

# get sens
def sens(v):
    perf = performance(v[1], v[0][:,1])
    return perf["sens"]

# get bal acc
def bal_acc(v):
    perf = performance(v[1], v[0][:,1], bal_acc=True)
    return perf["bal_acc"]

def flip_rate(k, v, n):
    preds = v[0]
    stop = preds.shape[0]
    assert stop%n == 0, f"Number of predictions ({stop}) is not evenly divisible by stepsize ({n})!"
    
    noise_seq = False
    if "noise" in k:
        noise_seq = True
        
    # calculate flip rate for a single image sequence (which contains 'n' number of frames)
    n_sum, m = 0, 0
    for i in range(0, stop, n):
        m += 1
        seq_preds = preds[i:i+n].argmax(dim=1)
        
        if noise_seq:
            n_sum += sum(seq_preds[0]!=seq_preds[1:])
        else:
            n_sum += sum(seq_preds[:-1]!=seq_preds[1:])
    
    return (n_sum/(m*(n-1))).item()

In [ ]:
# export

# get flib probability
def flip_prob(k, v, n):
    preds = v[0]
    stop = preds.shape[0]
    assert stop%n == 0, f"Number of predictions ({stop}) is not evenly divisible by stepsize ({n})!"
    
    noise_seq = False
    if "noise" in k:
        noise_seq = True
        
    # calculate flip rate for a single image sequence (which contains 'n' number of frames)
    n_sum, m = 0, 0
    for i in range(0, stop, n):
        m += 1
        seq_preds = preds[i:i+n].argmax(dim=1)
        
        if noise_seq:
            n_sum += sum(seq_preds[0]!=seq_preds[1:])
        else:
            n_sum += sum(seq_preds[:-1]!=seq_preds[1:])
    
    return (n_sum/(m*(n-1))).item()

def relative_perf(perf_c:dict, perf_cl:float):
    
    def subtract(v, x): return v-x
    
    apply_subtract = partial(subtract, x=perf_cl)
    return apply_to_vals(perf_c, apply_subtract)

def adjust_by_baseline(perf:dict, base:dict):
    
    perf = perf.copy()
    for tfm in perf.keys():
        perf[tfm] = perf[tfm]/base[tfm]
        
    return perf